In [1]:
# <수행 방법 요약>
# 이미지에서 각 모델들이 인식한 객체들에게서 바운딩된 박스, 신뢰도, 라벨들을 추출한다.
# 모든 박스들 중에서 신뢰도가 가장 높은 박스를 추출한다.
# 이미지에 추출한 박스의 좌표, 신뢰도, 라벨 이름을 표시한다.

from ultralytics import YOLO
import numpy as np
from torchvision.ops import nms
import torch
import cv2

# 모델 파일명 리스트
model_files = [f'models/model{i}.pt' for i in range(1, 13)]

# 모델 로드 및 names 저장
models = []
model_names = {}
class_name_mapping = {}

# 모든 모델의 클래스 통합
for i, model_file in enumerate(model_files):
    model = YOLO(model_file)
    models.append(model)
    model_names[f'model{i+1}'] = model.names
    for class_id, class_name in model.names.items():
        if class_name not in class_name_mapping:
            class_name_mapping[class_name] = []
        class_name_mapping[class_name].append((f'model{i+1}', class_id))

In [2]:
def ensemble_predict(image):
    results = []
    for model in models:
        results.append(model.predict(image, conf=0.5))
    combined_results = combine_results(*results)
    return combined_results

def combine_results(*results):
    combined_boxes = []
    combined_confidences = []
    combined_labels = []  # 각 박스의 라벨을 저장하기 위한 리스트

    # 각 모델의 결과에서 바운딩 박스를 추출하여 결합
    for model_index, result_list in enumerate(results):
        model_name = f'model{model_index + 1}'
        for result in result_list:
            for box in result.boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                conf = box.conf[0].cpu().numpy()
                class_id = int(box.cls[0].cpu().numpy())
                
                # 클래스 번호를 클래스 이름으로 변환
                class_name = model_names[model_name][class_id]
                
                combined_boxes.append([x1, y1, x2, y2])
                combined_confidences.append(conf)
                combined_labels.append(class_name)

    # NMS (Non-Maximum Suppression) 적용
    boxes = np.array(combined_boxes)
    # print('boxes:')
    # print(boxes)
    confidences = np.array(combined_confidences)

    indices = nms(torch.tensor(boxes), torch.tensor(confidences), 0.4)
    # print('indices:')
    # print(indices)
    final_boxes = boxes[indices]
    final_confidences = confidences[indices]
    
    # NMS 후 최종 박스와 일치하는 라벨 선택
    final_labels = [combined_labels[i] for i in indices]

    return final_boxes, final_confidences, final_labels

# 이미지 파일을 읽어오기
image_path = 'potato.jpg'
image = cv2.imread(image_path)

# 이미지에 대해 앙상블 예측 수행
combined_results = ensemble_predict(image)

# 결과 출력 및 이미지에 그리기
boxes, confidences, labels = combined_results

print("\ndetect:")
for box, conf, label in zip(boxes, confidences, labels):
    x1, y1, x2, y2 = map(int, box)
    label_name = label  # 이미 label은 클래스 이름입니다.

    # 레이블 이름이 확인되었으면, 이미지를 수정합니다.
    if label_name is not None:
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, f'{label_name} {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        print(f'\nBox: {box}, Confidence: {conf}, Label: {label_name}')

# 수정된 이미지 저장
output_image_path = 'potato_with_boxes.jpg'
cv2.imwrite(output_image_path, image)
print(f"\nAnnotated image saved as {output_image_path}")



0: 640x640 (no detections), 18.6ms
Speed: 2.5ms preprocess, 18.6ms inference, 13.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 potatos, 19.9ms
Speed: 1.0ms preprocess, 19.9ms inference, 39.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 potatos, 18.5ms
Speed: 1.5ms preprocess, 18.5ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 potatos, 18.5ms
Speed: 2.0ms preprocess, 18.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 Eggs, 19.1ms
Speed: 1.0ms preprocess, 19.1ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 17.9ms
Speed: 2.0ms preprocess, 17.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 potatos, 19.0ms
Speed: 2.1ms preprocess, 19.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 19.1ms
Speed: 1.0ms preprocess, 19.1ms inference, 1.0ms postprocess per